In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
import librosa
from skimage.color import gray2rgb
from skimage.transform import resize

# Load the trained model
model = load_model("bird_classifier_model.h5")

# Load the Label Encoder
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('label_encoder.npy', allow_pickle=True)

with open('max_width.txt', 'r') as file:
    max_width = int(file.read().strip())

# Function to extract Mel spectrograms from audio files
def extract_mel_spectrogram(audio_file, n_mels=64, hop_length=512):
    y, sr = librosa.load(audio_file, sr=None)
    y_high_pass = librosa.effects.preemphasis(y)
    mel_spectrogram = librosa.feature.melspectrogram(y=y_high_pass, sr=sr, n_mels=n_mels, hop_length=hop_length)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
    return mel_spectrogram

# Function to preprocess input audio file for prediction
def preprocess_input(audio_file):
    mel_spec = extract_mel_spectrogram(audio_file)
    # max_width = 313  
    pad_width = max_width - mel_spec.shape[1]
    padded_spec = np.pad(mel_spec, ((0, 0), (0, pad_width)), mode='constant')
    # Normalize spectrogram to range [0, 1]
    normalized_spec = (padded_spec - np.min(padded_spec)) / (np.max(padded_spec) - np.min(padded_spec))
    
    # Convert to RGB
    rgb_spec = gray2rgb(normalized_spec)
    resized_spec = resize(rgb_spec, (224, 224, 3))
    return np.expand_dims(resized_spec, axis=0)

# Function to predict bird species from audio file
def predict_bird_species(audio_file):
    # Preprocess input audio file
    processed_input = preprocess_input(audio_file)
    # Make prediction
    predictions = model.predict(processed_input)
    # Decode predicted label
    predicted_label = label_encoder.inverse_transform([np.argmax(predictions)])[0]
    return predicted_label

# Example usage
audio_file_path = "birds_audio_dataset\FRIGATE\XC619649 - Great Frigatebird - Fregata minor palmerstoni.mp3"
predicted_species = predict_bird_species(audio_file_path)
print("Predicted bird species:", predicted_species)
